In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import pymongo

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw
from climatools.cliradlw.utils import mapband_new2old

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *

import IPython.display as display

In [2]:
output_notebook()

Loading BokehJS ...

In [3]:
def make_query(param=None):
    return {'param.' + name: value for name, value in param.items()}   

In [4]:
def show_html(s): display.display(display.HTML(s))
    
def show_markdown(s): display.display(display.Markdown(s))

In [5]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['i', 'band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [6]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    dims = ['igg', 'g', 'i']
    for dim in dims:
        if dim in ds:
            if ds[dim].shape == (1,): ds = ds.squeeze(dim).drop(dim)
            elif ds[dim].shape == (): ds = ds.drop(dim)
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')     
    return ds['coolrg']

def pltdata_cooling_1():
    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(DS_COOL.sel(i=1)),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(DS_COOL_CRD.sum('g')),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(DS_COOL_WGT.sel(igg=10).sum('g')),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data

def nice_xlims(pltdata=None, prange=None):
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)

def plt_cool(pltdata=None):
    plin = plt_vert_profile_bokeh(pltdata=pltdata,
                                  y_axis_type='linear',
                                  prange=(50, 1050))
    plin.plot_width, plin.plot_height = 400, 500
    plog = plt_vert_profile_bokeh(pltdata=pltdata,
                                  y_axis_type='log',
                                  prange=(.01, 200))
    plog.plot_width, plog.plot_height = 400, 500
    everything = gridplot(plin, plog, ncols=2)
    return everything
    
def script_plt_cooling():
    pltdata = pltdata_cooling_1()
    p = plt_cool(pltdata=pltdata)
    show(p)
    show_markdown('''*FIGURE.* Cooling rate profiles.''')

In [7]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels: ds = ds.isel(pressure=ilevels)

    dims = ['i', 'igg', 'g']
    for dim in dims:
        if dim in ds:
            if ds[dim].shape == (1,): ds = ds.squeeze(dim).drop(dim)
            elif ds[dim].shape == (): ds = ds.drop(dim)
    
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df

def tbdata_flux():
    tropopauses = {'mls': 40, 'saw': 45, 'trp': 37}
    atmpro = PARAM['atmpro']
    ilevels = [0, tropopauses[atmpro], -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(DS_FLUX_CRD.sum('g'), ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10',
            'df': fmt_flux(DS_FLUX_WGT.sum('g').sel(igg=10), 
                           ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(DS_FLUX.sel(i=1), ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data

def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        show_markdown(f'''*TABLE.*  Fluxes. {d['label']}''')
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        show_markdown(
            f'''*TABLE.*  Fluxes.  ({d['label']}) - ({d0['label']})''')
        
    show_tb(benchmark)
    for d in others: show_tb(d)
    for d in others: show_tbdiff(d=d, d0=benchmark)
        
def script_tb_flux():
    data = tbdata_flux()
    tb_flux(tbdata=data)

In [8]:
def gasbands():
    bmap = mapband_new2old()
    gases = [gas for gas, _ in PARAM['molecule'].items()] 
    bands = [bmap[b] for b in PARAM['band']]
    return list(itertools.product(gases, bands))

def print_bestfit_params():
    df = pd.DataFrame()
    for gas, band in gasbands():
        param = bestfits.kdist_params(molecule=gas, band=band)
        srs = pd.Series(param)
        df[f'({gas}, {band})'] = srs
    df.fillna('-', inplace=True)
    pd.set_option('display.max_colwidth', -1)
    display.display(df.sort_index())
    pd.reset_option('display.max_colwidth')
        
def print_input_param():
    df_clirad = pd.Series(PARAM, name='clirad run').to_frame()
    df_lblnew = pd.Series(PARAM_LBLNEW, name='lblnew run compared against').to_frame()
    df = pd.merge(df_clirad, df_lblnew, how='outer', 
                  left_index=True, right_index=True)
    df.fillna('-', inplace=True)
    pd.set_option('display.max_colwidth', -1)
    display.display(df)
    pd.reset_option('display.max_colwidth')
        
def print_lblnew_param():
    pd.set_option('display.max_colwidth', -1)
    display.display(pd.Series(PARAM_LBLNEW).to_frame())
    pd.reset_option('display.max_colwidth')

In [9]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD.sum('g'), DS_COOL.sel(i=1)]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))

In [10]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD.sum('g'), DS_FLUX.sel(i=1)]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))

In [11]:
def script_fluxcoolr_excelfile():
    '''
    Save fluxes and cooling rates by layer by g-interval in excel file
    and make it available for download.
    '''
    fname = 'output_byg.xlsx'
    
    writer = pd.ExcelWriter(fname)
    
    for g in DS_FLUX_WGT.coords['g']:
        df = DS_FLUX_WGT.sel(igg=1).sel(g=g).to_dataframe()
        df.to_excel(writer, f'flux g={int(g)}')
        
    for g in DS_COOL_WGT.coords['g']:
        df = DS_COOL_WGT.sel(igg=1).sel(g=g).to_dataframe()
        df.to_excel(writer, f'cool g={int(g)}')    

In [12]:


    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()

In [13]:
%run param.py

client = pymongo.MongoClient('localhost', 27017)

doc = client.lblnew.bestfit_lw.find_one(make_query(param=PARAM_LBLNEW))
DS_FLUX_CRD = load_output_file(io.StringIO(doc['output_fluxg']))
DS_COOL_CRD = load_output_file(io.StringIO(doc['output_coolrg']))
DS_FLUX_WGT = load_output_file(io.StringIO(doc['output_wfluxg']))
DS_COOL_WGT = load_output_file(io.StringIO(doc['output_wcoolrg']))

doc = client.cliradnew.lw.find_one(make_query(param=PARAM))
DS_FLUX = load_output_file(io.StringIO(doc['output_flux']))
DS_COOL = load_output_file(io.StringIO(doc['output_coolr']))

In [14]:
script()
if 'ng_refs' in PARAM_LBLNEW:
    script_fluxcoolr_excelfile()

### CLIRAD-LW

### Table of Contents

# Input Parameters

,clirad run,lblnew run compared against
atmpro,trp,trp
band,[11],9
commitnumber,1013f91,5014a19
conc,-,-
dv,-,0.001
klin,-,1e-24
molecule,{'h2o': 'atmpro'},h2o
ng_adju,-,[0]
ng_refs,-,[5]
nv,-,1000


# Best-fit Parameters

...

*TABLE.*  Fluxes.  (CLIRAD) - (CRD)

# Cooling Rate by Layer

CRD               CLIRAD `coolr_bands`              
            layer        coolrg                layer        coolrg
pressure                                                          
0.000312        1 -4.319705e-04                    1 -1.990826e-05
0.000750        2 -4.312287e-04                    2 -2.267852e-05
0.001052        3 -4.292894e-04                    3 -1.612594e-05
0.001476        4 -4.271620e-04                    4 -1.497605e-05
0.002070        5 -4.248950e-04                    5 -1.291402e-05
0.002904        6 -4.223493e-04                    6 -1.163519e-05
0.004074        7 -4.196678e-04                    7 -9.634187e-06
0.005714        8 -4.169421e-04                    8 -7.754979e-06
0.008015        9 -4.141060e-04                    9 -5.935530e-06
0.011243       10 -4.114446e-04                   10 -3.802821e-06
0.015771       11 -4.088781e-04                   11 -1.819744e-06
0.022122       12 -4.067335e-04                   12  5.028527e-07
0.031031       13 -4.051619e-04                   13  2.604376e-06
0.043528       14 -4.032118e-04                   14  4.995433e-06
0.061057       15 -3.828951e-04                   15  1.724288e-05
0.085645       16 -3.151524e-04                   16  5.134057e-05
0.120136       17 -1.849579e-04                   17  1.116502e-04
0.168516       18  2.023407e-05                   18  2.038500e-04
0.236378       19  3.292719e-04                   19  3.421141e-04
0.331549       20  7.813866e-04                   20  5.473082e-04
0.465100       21  1.430542e-03                   21  8.480444e-04
0.652400       22  2.243104e-03                   22  1.232389e-03
0.915100       23  2.517171e-03                   23  1.393689e-03
1.283650       24  2.106559e-03                   24  1.243741e-03
1.800600       25  1.552072e-03                   25  1.007087e-03
2.525700       26  9.953958e-04                   26  7.440889e-04
3.542800       27  5.938715e-04                   27  5.306792e-04
4.969550       28  3.400874e-04                   28  3.707086e-04
6.970850       29  1.868379e-04                   29  2.520658e-04
9.778100       30  9.788299e-05                   30  1.637007e-04
13.715850      31  4.735139e-05                   31  9.851320e-05
19.239350      32  1.810607e-05                   32  4.997874e-05
26.987250      33 -2.586253e-07                   33  1.348291e-05
37.855300      34 -1.430668e-05                   34 -1.510939e-05
53.100050      35 -3.557504e-05                   35 -4.235827e-05
73.887500      36 -5.622243e-05                   36 -6.362639e-05
97.662500      37 -6.824655e-05                   37 -7.511985e-05
121.437500     38 -7.354455e-05                   38 -8.130219e-05
145.212500     39 -7.071632e-05                   39 -8.254693e-05
168.987500     40 -6.617717e-05                   40 -8.309196e-05
192.762500     41 -6.736176e-05                   41 -9.585160e-05
216.537500     42 -8.929885e-05                   42 -1.513651e-04
240.312500     43 -9.870020e-05                   43 -2.324342e-04
264.087500     44 -9.103735e-05                   44 -3.833512e-04
287.862500     45 -5.632961e-05                   45 -4.744341e-04
311.637500     46  2.286311e-05                   46 -3.627586e-04
335.412500     47  1.609539e-04                   47 -1.826602e-04
359.187500     48  3.599475e-04                   48 -1.820585e-04
382.962500     49  6.592809e-04                   49 -4.271416e-04
406.737500     50  1.150038e-03                   50 -5.821110e-04
430.512500     51  1.830619e-03                   51 -2.982308e-04
454.287500     52  2.713052e-03                   52  5.697941e-04
478.062500     53  3.809038e-03                   53  2.219080e-03
501.837500     54  5.124194e-03                   54  4.253937e-03
525.612500     55  6.675529e-03                   55  6.889309e-03
549.387500     56  8.459949e-03                   56  9.413376e-03
573.162500     57  1.050027e-02                   57  1.19692

# Fluxes by Layer

CRD                            CLIRAD                \
                       flug          fldg     fnetg      flug          fldg   
pressure    level                                                             
0.000000    1     -5.975937  0.000000e+00 -5.975937 -5.604940  7.146931e-09   
0.000624    2     -5.975937  4.658226e-09 -5.975937 -5.604940  8.535673e-09   
0.000876    3     -5.975937  6.574360e-09 -5.975937 -5.604940  9.242431e-09   
0.001229    4     -5.975937  9.378620e-09 -5.975937 -5.604940  1.037283e-08   
0.001723    5     -5.975937  1.348593e-08 -5.975937 -5.604940  1.189946e-08   
0.002417    6     -5.975937  1.949963e-08 -5.975937 -5.604940  1.402718e-08   
0.003391    7     -5.975937  2.830448e-08 -5.975937 -5.604940  1.699312e-08   
0.004757    8     -5.975938  4.119426e-08 -5.975937 -5.604940  2.123292e-08   
0.006672    9     -5.975938  6.003845e-08 -5.975938 -5.604940  2.730217e-08   
0.009359    10    -5.975938  8.759353e-08 -5.975938 -5.604940  3.603899e-08   
0.013128    11    -5.975938  1.278486e-07 -5.975938 -5.604941  4.872215e-08   
0.018415    12    -5.975938  1.865784e-07 -5.975938 -5.604941  6.712611e-08   
0.025830    13    -5.975939  2.721806e-07 -5.975939 -5.604941  9.399670e-08   
0.036232    14    -5.975939  3.967594e-07 -5.975939 -5.604941  1.331506e-07   
0.050823    15    -5.975940  5.798636e-07 -5.975940 -5.604941  1.912755e-07   
0.071291    16    -5.975942  8.870841e-07 -5.975941 -5.604941  2.917563e-07   
0.100000    17    -5.975943  1.501470e-06 -5.975942 -5.604941  4.989137e-07   
0.140271    18    -5.975945  2.788380e-06 -5.975943 -5.604941  9.451375e-07   
0.196760    19    -5.975948  5.444128e-06 -5.975942 -5.604940  1.896094e-06   
0.275997    20    -5.975950  1.083457e-05 -5.975939 -5.604939  3.903154e-06   
0.387100    21    -5.975951  2.158755e-05 -5.975929 -5.604936  8.102983e-06   
0.543100    22    -5.975945  4.268558e-05 -5.975903 -5.604929  1.682739e-05   
0.761700    23    -5.975926  8.177503e-05 -5.975845 -5.604914  3.407299e-05   
1.068500    24    -5.975891  1.382140e-04 -5.975753 -5.604891  6.118110e-05   
1.498800    25    -5.975847  2.008989e-04 -5.975646 -5.604862  9.526045e-05   
2.102400    26    -5.975796  2.615357e-04 -5.975535 -5.604829  1.344070e-04   
2.949000    27    -5.975746  3.112505e-04 -5.975435 -5.604795  1.758079e-04   
4.136600    28    -5.975699  3.472095e-04 -5.975351 -5.604763  2.184442e-04   
5.802500    29    -5.975654  3.699254e-04 -5.975284 -5.604734  2.619187e-04   
8.139200    30    -5.975614  3.811638e-04 -5.975233 -5.604708  3.058013e-04   
11.417000   31    -5.975578  3.835961e-04 -5.975194 -5.604688  3.493921e-04   
16.014700   32    -5.975549  3.802985e-04 -5.975169 -5.604677  3.918980e-04   
22.464000   33    -5.975529  3.742491e-04 -5.975155 -5.604679  4.321252e-04   
31.510500   34    -5.975523  3.678629e-04 -5.975155 -5.604701  4.684834e-04   
44.200100   35    -5.975539  3.619276e-04 -5.975177 -5.604753  4.983502e-04   
62.000000   36    -5.975600  3.486464e-04 -5.975252 -5.604857  5.133020e-04   
85.775000   37    -5.975737  3.266875e-04 -5.975410 -5.605031  5.077670e-04   
109.550000  38    -5.975916  3.133188e-04 -5.975602 -5.605230  4.945211e-04   
133.325000  39    -5.976133  3.240816e-04 -5.975809 -5.605454  4.894543e-04   
157.100000  40    -5.976388  3.796287e-04 -5.976008 -5.605707  5.103419e-04   
180.875000  41    -5.976695  5.000453e-04 -5.976195 -5.606008  5.771527e-04   
204.650000  42    -5.977156  7.710858e-04 -5.976385 -5.606462  7.610807e-04   
228.425000  43    -5.978173  1.536701e-03 -5.976636 -5.607491  1.364153e-03   
252.200000  44    -5.980131  3.217359e-03 -5.976914 -5.609514  2.732603e-03   
275.975000  45    -5.983536  6.365731e-03 -5.977170 -5.613093  5.231403e-03   
299.750000  46    -5.989003  1.167386e-02 -5.977329 -5.618908  9.710217e-03   
323.525000  47    -5.997246  1.998124e-02 -5.977265 -5.627737  1.751768e-02   
347.300000  48    -6.009073  3.226177e-02 -5.976811 -5.640566  2.983243e-0

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')